## Note: the visualzations in this notebook do not work currently

The brunel visualization library does not yet support Spark 2.0. Support for Spark 2.0 is upcoming
with the final release of brunel 2.2; the notebook will be updated when this is available

If you are new to dashDB and it's integrated Apache Spark capabilities you may first want to check out this [overview article](http://www.ibmbigdatahub.com/blog/evolving-enterprise-data-warehouse-beyond-sql-apache-spark). A life walkthough of this demo notebook can be found on [youtube](https://youtu.be/ZPYkWVQw1gg).

## Demo setup

In [1]:
/*
 * © Copyright IBM Corporation 2016 All rights reserved.
 * LICENSE: BSD-3, https://opensource.org/licenses/BSD-3-Clause
 */

import org.apache.spark.ml.linalg.{Vector, Vectors}
import org.apache.spark.sql.SQLContext
import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.types.DoubleType
import org.apache.spark.sql.types.StructType
import org.apache.spark.sql.types.StructField
import org.apache.spark.sql.Row
import org.apache.spark.sql.types.IntegerType
import java.sql.Connection
import java.sql.SQLException
import java.sql.ResultSet
import java.sql.Statement
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.clustering.KMeans
import org.apache.spark.ml.clustering.KMeansModel
import org.apache.spark.ml.feature.VectorAssembler

Load the brunel visualitzation lib for Scala. <b>Note</b> that the brunelvis.org server is often not accessible from certain locations, e.g. from softlayer machines. In that case download the library by hand and upload to dashDB via __`spark-submit.sh --upload-files`__.

In [2]:
//NOT-FOR-APP
//Load the library directly from the repository:
%AddJar -magic https://brunelvis.org/jar/spark-kernel-brunel-all-2.2.jar

//Use the following lines (adapt user name) if access to repository is not working due to some local firewall constraints.
//%AddJar -magic file:/mnt/blumeta0/home/bluadmin/spark/apps/spark-kernel-brunel-all-2.2.jar -f

Starting download from https://brunelvis.org/jar/spark-kernel-brunel-all-2.2.jar
Finished download of spark-kernel-brunel-all-2.2.jar


Name: Syntax Error.
Message: 
StackTrace: 

The following cell prepares the input data based on sample tables that are available in every dashDB system out of the box. So you can run this notebook against any dashDB instance that you have available. But when the cell runs for the first time the setup can take a couple of minutes.

In [3]:
//NOT-FOR-APP
import java.sql.DriverManager
import java.sql.Connection
val connection = DriverManager.getConnection("jdbc:db2:BLUDB")
try {   
    val statement = connection.createStatement()
    statement.executeUpdate("create table tornado as ( " +
         " select objectid,yr,mo,dy, time, cast(slat as float) as slat,cast(slon as float) as slon, " +
         "        elat,elon,cast (len as float) as len,cast(wid as float) as wid,st,cast(mag as float) as mag " +
         " from samples.geo_tornado ) with data")
} catch {
    case e: SQLException => { if (e.getErrorCode() == -601) println("Table TORNADO already exists") }
}
try {   
    val statement = connection.createStatement()
    statement.executeUpdate("create table texas_customers as ( " +
         " select b.objectid, b.name, b.insurance_value, db2gse.st_x(b.shape) lon, db2gse.st_y(b.shape) lat " +
         "     from samples.geo_county a, samples.geo_customer b " +
         "     where a.statefp='48' and db2gse.st_within(b.shape, a.shape) = 1 " +
         " ) with data")
} catch {
    case e: SQLException => { if (e.getErrorCode() == -601) println("Table TEXAS_CUSTOMERS already exists") }
}
connection.close()

Table TORNADO already exists
Table TEXAS_CUSTOMERS already exists


## Data acquisition

Read the training data using the IDAX data source. In case of dashDB MPP systems this will give us
automatically a DataFrame with exactly the number of partitions as there are MLNs (database partitions)
in dashDB and the DataFrame partitions will be located in each according executor colocated with the
dashDB MLN.

In [4]:
var input = spark.read.
   format("com.ibm.idax.spark.idaxsource").
   option("dbtable", "TORNADO").
   load()

The input data are all recorded tornados in USA since 1950. It is taken from http://www.spc.noaa.gov/.

When this demo is run with a dashDB SMP system (i.e. non partitioned databse) the next step should be run in order to achieve parallel computation. You should skip this step in case you run against a dashDB MPP because then the data frame is partitioned automatically for you.

In [5]:
// (4) Repartition to 10 partitions
// not needed for MPP
input = input.repartition(10)

## Data exploration

Let's take a look at the Tornado data now:

In [6]:
//NOT-FOR-APP
input.show()

val density = input.select("ST").groupBy("ST").count()
density.orderBy(org.apache.spark.sql.functions.col("count").desc).show()


+--------+----+---+---+--------+-----+-------+-----+-------+----+-----+---+----+
|OBJECTID|  YR| MO| DY|    TIME| SLAT|   SLON| ELAT|   ELON| LEN|  WID| ST| MAG|
+--------+----+---+---+--------+-----+-------+-----+-------+----+-----+---+----+
|    3540|1957| 11|  7|23:30:00|30.35|  -92.2|30.37| -92.07| 9.2|200.0| LA| 3.0|
|     347|1951|  6| 17|23:00:00|38.62| -95.08|  0.0|    0.0| 0.1| 10.0| KS|-9.0|
|     357|1951|  6| 21|01:00:00|39.37| -98.88| 39.4| -98.47|22.2| 10.0| KS|-9.0|
|     367|1951|  6| 25|15:30:00| 41.1|-103.05|41.22|-102.92|10.4| 57.0| NE| 2.0|
|     377|1951|  6| 27|20:45:00|40.27| -89.47|40.27| -89.12|18.2|200.0| IL| 3.0|
|     387|1951|  7|  8|20:15:00|39.22| -94.02|39.25| -94.02| 2.3|100.0| MO| 2.0|
|     397|1951|  7| 21|11:00:00| 41.8| -76.47|  0.0|    0.0| 0.1| 10.0| PA| 0.0|
|     407|1951|  8| 10|13:57:00| 35.8|  -97.2|35.88| -97.17| 5.9|200.0| OK| 1.0|
|     417|1951|  8| 21|14:00:00| 42.7| -71.13|42.77| -70.97| 9.3|100.0| MA| 2.0|
|     427|1951|  8| 31|16:20

For visualization we sample the data down a bit:

In [7]:
//NOT-FOR-APP
val inputSample = input.sample(false,0.1)

Now let's visualize all sampled tornado start coordinates on a map:

In [9]:
%%brunel map('usa') + data('inputSample') x(slon) y(slat) title("Input Data Density per State") :: width=800, height=400
        

Magic brunel failed to execute with error: 
org.apache.spark.sql.DataFrame

Let's visualize tht as a density map summed up per state:

In [8]:
%%brunel map('usa') + data('density') map color(count:[blues, reds]) key(st) label(st) title("Input Data Density per State")  :: width=800, height=800

Let's focus on Texas as the state with highest density of tornados. So let's filter the tornado records for Texas only and see how many we have:

In [8]:
val input_texas = input.filter("ST = 'TX'")
input_texas.count()

8197

## Machine Learning
Let's use machine learning to cluster the hot spots for tornados within Texas. We use a classical clustering ML algorithm.

First we define the feature selection stage of the Spark ML pipeline. We train based on Tornado latitude and logitude.

In [9]:
val assembler = new VectorAssembler().
            setInputCols(Array("SLAT", "SLON")).
            setOutputCol("features")

Now we define the model training stage for unsupervised cluster training using KMeans algorithm from Spark ML API. You can change the number of iterations as well as the number of cluster centers that should be detected:

In [10]:
val clustering = new KMeans().
            setFeaturesCol("features").
            setK(5).
            setMaxIter(5).
            setSeed(123)

Now we stitch together the ML pipeline with feature selection and training stages:

In [11]:
val pipe = new Pipeline().setStages(Array(assembler, clustering))

Now run the ML pipeline on the input data of Texas tornados:

In [12]:
val model = pipe.fit(input_texas)

This ran quick. If you want to observe a longer model training run, execute the following cell:

In [15]:
var input_ballooned = input.unionAll(input).unionAll(input).unionAll(input).unionAll(input).unionAll(input).unionAll(input)
pipe.fit(input_ballooned)

pipeline_8fc8aeb3a90e

Let's explore the trained model and print out the detected cluster centers with their latitude and lognitude:

In [14]:
//NOT-FOR-APP
val clustercenters = model.stages(1).asInstanceOf[KMeansModel].clusterCenters
val clustercentersschema = StructType(for (i <- 1 to clustercenters.head.size) yield StructField(s"col$i", DoubleType, false))
val clustercentersconverted = clustercenters.map { x: Vector => Row.fromSeq(x.toArray) }
val clustercentersdataframe = spark.createDataFrame(sc.parallelize(clustercentersconverted), clustercentersschema).toDF("SLAT", "SLON")
clustercentersdataframe.show()

+------------------+-------------------+
|              SLAT|               SLON|
+------------------+-------------------+
|32.709330515257626| -96.89806503251627|
|34.754479642293525|-101.45829900052604|
| 28.78481337973705| -97.81904276875481|
| 32.07568572364112|-101.03350812049771|
| 30.43746912660799| -95.06054238320922|
+------------------+-------------------+



Let's plot that on a map:

In [15]:
%%brunel map('usa') + data('clustercentersdataframe') x(slon) y(slat)  title("Tornado Cluster Centers")

Comparing to official NOAA Tornado density maps confirms that our cluster centers make sense.

![title](http://www.spc.noaa.gov/wcm/2015-wbc-anoms.png)

Now we test the model by simply assigning (scoring) the clusters for each recorded tornado in Texas and then aggregating and ranking each cluster by the number of assigned tornados:

In [15]:
//NOT-FOR-APP
val output_train = model.transform(input_texas).select("OBJECTID","SLON","SLAT","prediction")
val tornados_with_clusters = output_train.toDF(Seq("OBJECTID","SLON","SLAT","ClusterID"): _*)

tornados_with_clusters.groupBy("ClusterID").count().orderBy(org.apache.spark.sql.functions.col("count").desc).show()

+---------+-----+
|ClusterID|count|
+---------+-----+
|        0| 1982|
|        1| 1897|
|        3| 1521|
|        4| 1500|
|        2| 1297|
+---------+-----+



## Making some real predictions

We want to assign each holder of an insurance policies for real estate in Texas to one of the tornado clusters in order to caclulate a risk score according to which cluster his property belongs to.

First we read in insurance holder records for Texas (this is mock up data for the sake of this demo notebook):

In [17]:
var insurance_holders = spark.read.
   format("com.ibm.idax.spark.idaxsource").
   option("dbtable", "TEXAS_CUSTOMERS").
   load()
insurance_holders = insurance_holders.repartition(10)

Now let's take a look at the insurance holder data:

In [18]:
//NOT-FOR-APP
var insurance_holders_sample = insurance_holders.sample(false,0.05)
insurance_holders.show()
insurance_holders.count()

+--------+-------------------+---------------+-------------------+------------------+
|OBJECTID|               NAME|INSURANCE_VALUE|                LON|               LAT|
+--------+-------------------+---------------+-------------------+------------------+
|  362854|      Pat Steinbeck|         235686| -97.66384647023835| 30.97831721167006|
|  371703|       Elois Monier|         245913| -98.05744696660418| 27.35959492836332|
|  371773|Carlita Fitzsimmons|         499337| -98.10237384137963| 27.53699321486505|
|  371789|     Carmela Casias|         215623| -97.96883522216774|27.780208215336927|
|  371659|         Garth Hine|        1282961|  -97.8052835507952|27.452977658092493|
|  371502|     Delia Sinclair|         188756| -97.92067983970281|27.425224045553307|
|  361060|       Wally Stupka|        2220953| -97.66137135970047|32.332674550190404|
|  361019|      Elisha Clauss|         248175| -98.00661729265391| 32.43711119342686|
|  360847|   Charlyn Dashiell|         245537| -97.620

83663

Let's visualize a sample of the Texas insurance holders on a map:

In [19]:
%%brunel map('usa') + data('insurance_holders_sample') x(lon) y(lat) title("Customers in Texas") :: width=800, height=400 

Finally we run a prediction with the tornado cluster model on the insurance holders to assign each one to its according tornado cluster. Then we print a sample of the result.

In [20]:
val insurance_holders_renamed = insurance_holders.toDF(Seq("OBJECTID", "NAME", "INSURANCE_VALUE", "SLON", "SLAT"): _*)
var insurance_holders_scored = model.transform(insurance_holders_renamed).select("OBJECTID", "NAME", "INSURANCE_VALUE", "SLON","SLAT","prediction")
insurance_holders_scored = insurance_holders_scored.toDF(Seq("OBJECTID", "NAME", "INSURANCE_VALUE", "SLON", "SLAT", "ClusterID"): _*)

insurance_holders_scored.show(10)
//insurance_holders_scored.groupBy("ClusterID").count().show()

println("Writing results to table TEXAS_CUSTOMERS_SCORED ...")
insurance_holders_scored.write.format("com.ibm.idax.spark.idaxsource").
   option("dbtable", "TEXAS_CUSTOMERS_SCORED").
   mode(SaveMode.Overwrite).
   save()
println("Done")

+--------+----------------+---------------+-------------------+------------------+---------+
|OBJECTID|            NAME|INSURANCE_VALUE|               SLON|              SLAT|ClusterID|
+--------+----------------+---------------+-------------------+------------------+---------+
|     600|    Chery Pinedo|         106788|-100.53338535295165|29.973622900976125|        3|
|     610|  Suzan Triplett|         140730|-100.22630795075128|30.091558595652714|        3|
|    1117|     Alysha Song|         143913| -94.92238938226416|31.460897053955932|        4|
|    1127|Casimira Service|         199051| -94.68942673135854|31.653304376995436|        4|
|    1206|    Sara Deckman|         199746| -94.86873213008369| 31.77158673424043|        4|
|    1155| Dominga Burnley|        1534151| -94.61301162938315|31.476935394253076|        4|
|    1145|      Loura Bera|        1610900|  -94.7703494782458|31.225382249391373|        4|
|    1233|     Adolfo Huth|         105200|  -94.7412518567877|31.6084

The resulting cluster ID determines the risk class that the insurance holder will be assigned to. In addition we could run a fine tuning for the risk assignment by folding in the distance from the insured property to the cluster center.

Let's colorize the sampled Texas insurance holders according to their assigned cluster:

In [21]:
//NOT-FOR-APP
var insurance_holders_scored_sample = insurance_holders_scored.sample(false, 0.05)

In [22]:
%%brunel map('usa') + data('insurance_holders_scored_sample') x(SLON) y(SLAT) title("Texas Customers - Cluster-Relation") color(ClusterID) :: width=800, height=400 

## Deploying as application
You can easily deploy the essential logic of this notebook as an application that can be run at any time via a dashDB [stored procedure](https://www.ibm.com/support/knowledgecenter/SS6NHC/com.ibm.swg.im.dashdb.analytics.doc/doc/r_spark_applications_functions.html) invokation, via dashDB's command line [spark-submit.sh](https://www.ibm.com/support/knowledgecenter/SS6NHC/com.ibm.swg.im.dashdb.doc/learn_how/spark_ref.html) or via a [REST API](https://developer.ibm.com/clouddataservices/wp-content/themes/projectnext-clouddata/dashDBanalytics/#/).

Note that a few cells above have the __`//NOT-FOR-APP`__ comment. Cells with this comment as well as cells that only contain Jupyter cell magic starting with __`%%`__ are automatically skipped when deploying the notebook as an application. This way you can organize your notebook for interactive vs. backend logic.

In order to generate and deploy an application for this notebook select __`File->Deploy as->Deploy to dashDB Spark`__ in the menu above. After a short while it shows you the result page for the deployment with the specific invocation options that you just can copy and past to try them out immediately.